In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

## 1.
Dado un conjunto de datos de una dimensión $X = {-5.0, 23.0, 17.6, 7.23, 1.11 }$ normalizar usando:
1. Min_max normalizacion en el Intervalo [-1,1]
2. Desviación estandar
3. Escala decimal en el intervalo [-1,1]
4. Comparar los resultados y discutir ventajas y desventajas.

## 2.
Dado un conjunto de datos de 4 dimensiones con valores perdidos:
| I1 | I2 | I3 | I4 |
|:--:|:--:|:--:|:--:|
|  0 |  1 |  1 |  2 |
|  2 |  1 |  ? |  1 |
|  1 |  ? |  ? |  0 |
|  ? |  2 |  1 |  ? |
|  2 |  2 |  1 |  0 |

1. Dado que el dominio para todos los atributos es [0,1,2] ¿Cuál debe ser el número de ejemplos
“artificiales” si los valores perdidos son interpretados como “no importa el valor” y ellos son
remplazados con todos los posibles valores para su dominio.
2. ¿cuál otro método utilizaría para remplazar los valores pérdidos?

### Desarrollo
1. Si los valores perdidos son cualquiera de los valores perdidos marcados como '?' se toman como cualquier valor en el rango de $[0,1,2]$ los valores "artificales" serían 4. Ser vería por ejemplo así el nuevo dataset:

In [34]:
# dejar una semilla para siempre obtener el mismo resultado
random.seed(1)
# crear data frame
df = pd.DataFrame([[0,1,1,2],[2,1,'?',1],[1,'?','?',0],['?',2,1,'?'],[2,2,1,0]],columns=['I1','I2','I3','I4'])
# remplazar con un valor aleatorio de entre 0, 1 y 2
df.replace('?',random.choice([0,1,2]),inplace=True)
df

,I1,I2,I3,I4
0,0,1,1,2
1,2,1,0,1
2,1,0,0,0
3,0,2,1,0
4,2,2,1,0


2. Se podría usar la media de ese atributo por ejemplo, como se ve a continuacion :

In [20]:
df = pd.DataFrame([[0,1,1,2],[2,1,'?',1],[1,'?','?',0],['?',2,1,'?'],[2,2,1,0]],columns=['I1','I2','I3','I4'])
# remplazar con un valor aleatorio de entre 0, 1 y 2
df.replace('?',np.nan,inplace=True)
means = df.mean()
means

I1    1.25
I2    1.50
I3    1.00
I4    0.75
dtype: float64

In [21]:
df.replace(np.nan,means)

,I1,I2,I3,I4
0,0.00,1.0,1.0,2.00
1,2.00,1.0,1.0,1.00
2,1.00,1.5,1.0,0.00
3,1.25,2.0,1.0,0.75
4,2.00,2.0,1.0,0.00


También puede ser con la moda

In [45]:
df = pd.DataFrame([[0,1,1,2],[2,1,'?',1],[1,'?','?',0],['?',2,1,'?'],[2,2,1,0]],columns=['I1','I2','I3','I4'])
# remplazar con un valor aleatorio de entre 0, 1 y 2
df.replace('?',np.nan,inplace=True)
modes = df.mode()
modes

,I1,I2,I3,I4
0,2.0,1.0,1.0,0.0
1,NaN,2.0,NaN,NaN


La moda de I1 es 2, la de I2 puede ser 1 or 2, la moda de I3 es 1 y la de I4 0

In [46]:
df.replace(np.nan,modes.iloc[0])

,I1,I2,I3,I4
0,0.0,1.0,1.0,2.0
1,2.0,1.0,1.0,1.0
2,1.0,1.0,1.0,0.0
3,2.0,2.0,1.0,0.0
4,2.0,2.0,1.0,0.0


## 3.
El número de hijos de diferentes pacientes es dado por el siguiente vector:$
C = {3, 1, 0, 2, 7, 3, 6, 4, −2, 0, 0, 10, 15, 6}$
1. Encontrar “outliers” usando parámetros estadisticos estándar: media y varianza
    
    a. Si el umbral cambia de ±3 desviaciones estandar a ±2 desviaciones estandar , ¿Cuál
    “outlier adicional” se encuentra?

## 4. 
Dado un conjunto de tres dimensiones,
X=[{1,2,0},{3,1,4},{2,1,5},{0,1,6},{2,4,3},{4,4,2},{5,2,1},{7,7,7},{0,0,0},{3,3,3}]
1. Describir el procedimiento e interpretar los resultados de detección de outliers basado en la
media y varianza

In [107]:
df = pd.DataFrame([[1,2,0],[3,1,4],[2,1,5],[0,1,6],[2,4,3],[4,4,2],[5,2,1],[7,7,7],[0,0,0],[3,3,3]],columns=['x1','x2','x3'])
df

,x1,x2,x3
0,1,2,0
1,3,1,4
2,2,1,5
3,0,1,6
4,2,4,3
5,4,4,2
6,5,2,1
7,7,7,7
8,0,0,0
9,3,3,3


Se calcula la media para cada dimension

In [51]:
means = df.mean()
means

x1    2.7
x2    2.5
x3    3.1
dtype: float64

Luego las varianzas para cada dimension

In [112]:
variances = df.var()
variances

x1    4.900000
x2    4.277778
x3    5.877778
dtype: float64

Se usa el método apply para aplicar a todas las columnas una mascara que remplaza por NAN aquellos valores que son outliers, es decir que sean mayores que media+2*varianza o menores que media-2*varianza. 

In [111]:
def func(col):
    mean = means[col.name]
    variance = variances[col.name]
    col_new = col.mask(col<=mean-2*variance,np.nan)
    col_new = col.mask(col_new>=mean+2*variance,np.nan)
    return col_new

df.apply(func,axis=0)

,x1,x2,x3
0,1,2,0
1,3,1,4
2,2,1,5
3,0,1,6
4,2,4,3
5,4,4,2
6,5,2,1
7,7,7,7
8,0,0,0
9,3,3,3


Al parecer en este dataset no se reconocieron outliers

## 5. 
En Weka cargar el conjunto de datos iris.
1. Eliminar manualmente valores (15 %) en sus atributos, para simular valores perdidos. Luego
aplicar varios métodos que estan en weka para remplazar esos valores perdidos. Discutir las
diferencias entre el valor real y el que valor que lo remplaza, y las diferencias entre los métodos.
2. Normalizar usando varios metodos.
3. Discretizar usando varios métodos

## 6.
Dado el conjunto de datos:
| I1 |  I2 |  I3 |
|:--:|:---:|:---:|
|  1 | 5.9 | 3.4 |
|  2 | 2.1 | 6.2 |
|  1 | 1.6 | 2.8 |
|  2 | 6.8 | 5.8 |
|  1 | 3.1 | 3.1 |
| 1  | 8.3 | 4.1 |
| 2  | 2.4 | 5.0 |

Realizar reducción de valores basado en la técnica de BIN con el mejor corte para lo siguiente
(mostrar pasos):
1. Dimensión I2 usando la media como representantes de 2 BINS
2. Dimensión I3 usando el limite más cercano como representante de 2 BINS

1. primero creamos un dataframe y se ordena de acuerdo a los valores de la variable 2. 

In [195]:
df = pd.DataFrame([[1, 5.9 ,3.4],[ 2 ,2.1,6.2 ],[ 1 , 1.6 , 2.8 ],[ 2 , 6.8 , 5.8 ],[1 ,3.1 , 3.1],[1  , 8.3 , 4.1],[2  , 2.4 ,5.0]],columns=['I1','I2','I3'])
df

,I1,I2,I3
0,1,5.9,3.4
1,2,2.1,6.2
2,1,1.6,2.8
3,2,6.8,5.8
4,1,3.1,3.1
5,1,8.3,4.1
6,2,2.4,5.0


In [196]:
df.sort_values(by='I2',inplace=True,ignore_index=True)
df

,I1,I2,I3
0,1,1.6,2.8
1,2,2.1,6.2
2,2,2.4,5.0
3,1,3.1,3.1
4,1,5.9,3.4
5,2,6.8,5.8
6,1,8.3,4.1


Para decidir si 3.1 va en el primer bin o en el segundo, miramos la distancia al ultimo valor del bin 1 ( d= 0.7 )y el primer valor del bin 2 ( d= 2.8 ), ya que la distancia al ultimo valor del primer bin es menor, lo asignamos a ese bin. 

In [197]:
df_bin1 = df.iloc[:4,:].copy()
df_bin1

,I1,I2,I3
0,1,1.6,2.8
1,2,2.1,6.2
2,2,2.4,5.0
3,1,3.1,3.1


In [198]:
df_bin2 = df.iloc[4:,:].copy()
df_bin2

,I1,I2,I3
4,1,5.9,3.4
5,2,6.8,5.8
6,1,8.3,4.1


Ya que queremos usar la media para discretizar, calculamos la media para cada bin y reemplazamos los valores en el dataset original, asi obtendriamos valroes discretizados para la variable I2

In [199]:
bin_1_mean = df_bin1.mean()
bin_2_mean = df_bin2.mean()

In [200]:
df['I2']= df.index.map(lambda x: bin_1_mean['I2'] if x<4 else bin_2_mean['I2'])

In [201]:
df

,I1,I2,I3
0,1,2.3,2.8
1,2,2.3,6.2
2,2,2.3,5.0
3,1,2.3,3.1
4,1,7.0,3.4
5,2,7.0,5.8
6,1,7.0,4.1


2. Para discretizar la variable I3, ordenamos el dataset de acuerdo a los valores presentes en I3

In [202]:
df.sort_values(by='I3',inplace=True,ignore_index=True)
df

,I1,I2,I3
0,1,2.3,2.8
1,1,2.3,3.1
2,1,7.0,3.4
3,1,7.0,4.1
4,2,2.3,5.0
5,2,7.0,5.8
6,2,2.3,6.2


De nuevo el valor 4.1 es más cercano a 3.4 por lo que se asigna al bin 1. 

In [214]:
df_bin1 = df.iloc[:4,:].copy()
df_bin1

,I1,I2,I3
0,1,2.3,2.8
1,1,2.3,3.1
2,1,7.0,3.4
3,1,7.0,4.1


In [219]:
df_bin2 = df.iloc[4:,:].copy()
df_bin2

,I1,I2,I3
4,2,2.3,5.0
5,2,7.0,5.8
6,2,2.3,6.2


Para asignar los valores tomamos los extremos, 2.8 y 4.1 en el caso del bin 1 y si el valor es más cercano a 2.8 se asigna ese valor, de lo contrario se asigna 4.1. Lo mismo se haría para el bin 2, solo que los extremos tienen valroes 5 y 6.2

In [215]:
df_bin1['I3'] = df_bin1['I3'].apply(lambda x: 2.8 if abs(x-2.8)<abs(x-4.1) else 4.1 )

In [216]:
df_bin1

,I1,I2,I3
0,1,2.3,2.8
1,1,2.3,2.8
2,1,7.0,2.8
3,1,7.0,4.1


In [220]:
df_bin2['I3'] = df_bin2['I3'].apply(lambda x: 5 if abs(x-5)<abs(x-6.2) else 6.2 )

In [221]:
df_bin2

,I1,I2,I3
4,2,2.3,5.0
5,2,7.0,6.2
6,2,2.3,6.2


Finalmente se une el resultado

In [223]:
df_total = pd.concat([df_bin1,df_bin2])
df_total

,I1,I2,I3
0,1,2.3,2.8
1,1,2.3,2.8
2,1,7.0,2.8
3,1,7.0,4.1
4,2,2.3,5.0
5,2,7.0,6.2
6,2,2.3,6.2


## 7. 
Dado el conjunto de datos con tres dimensiones de entrada y una dimension representando la clase:
|  I1 |  I2 |  I3 | C |
|:---:|:---:|:---:|---|
| 2.5 | 1.6 | 5.9 | 0 |
| 7.2 | 4.3 | 2.1 | 1 |
| 3.4 | 5.8 | 1.6 | 1 |
| 5.6 | 3.6 | 6.8 | 0 |
| 4.8 | 7.2 | 3.1 | 1 |
| 8.1 | 4.9 | 8.3 | 0 |
| 6.3 | 4.8 | 2.4 | 1 |

Hacer el ranking de las dimensiones realizando comparación de medias y varianzas

## 8.
Dado el conjunto de datos X, donde X1 y X2 son dimensiones numericas, X3 y X4 son dimensiones
con datos categoricos 

|  X1 |  X2 | X3 | X4 |
|:---:|:---:|:--:|----|
| 2.7 | 3.4 |  1 | A  |
| 3.1 | 6.2 |  2 | A  |
| 4.5 | 2.8 |  1 | B  |
| 5.3 | 5.8 |  2 | B  |
| 6.6 | 3.1 |  1 | A  |
| 5.0 | 4.1 | 2  | B  |

1. Aplicar el método se selección de características basado en la entropía para reducir una di-
mensión (mostrar pasos).

2. Implementar un programa para realizar el “ranking” de dimensiones usando entropía.

Primero escribimos los datos en un dataframe

In [2]:
df = pd.DataFrame([[2.7,3.4,'1','A'],[3.1,6.2,'2','A'],[4.5,2.8,'1','B'],[5.3,5.8,'2','B'],[6.6,3.1,'1','A'],[5.0,4.1,'2','B']],columns=['X1','X2','X3','X4'])
df

,X1,X2,X3,X4
0,2.7,3.4,1,A
1,3.1,6.2,2,A
2,4.5,2.8,1,B
3,5.3,5.8,2,B
4,6.6,3.1,1,A
5,5.0,4.1,2,B


separamos las variables en atributos numéricos y categoricos ya que la medida de similitud es distinta dependiendo del tipo de variable

In [3]:
df_numerica = df.iloc[:,:2].copy()
df_numerica

,X1,X2
0,2.7,3.4
1,3.1,6.2
2,4.5,2.8
3,5.3,5.8
4,6.6,3.1
5,5.0,4.1


In [4]:
df_categorica = df.iloc[:,2:].copy()
df_categorica

,X3,X4
0,1,A
1,2,A
2,1,B
3,2,B
4,1,A
5,2,B


In [78]:
def similitud_categorica(df):
    matrix_ = df.to_numpy()
    n = matrix_.shape[0]
    m = matrix_.shape[1]
    similitud_categorica =np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            similitud_categorica[i][j]=(matrix_[i]==matrix_[j]).sum()/m
    return similitud_categorica

def similitud_numerica(df):
    n = df.shape[0]
    range_ = df_numerica.max()-df_numerica.min()
    distancia =np.zeros((n,n))
    D = 0
    for i in range(n):
        for j in range(n):
            diff = (df.loc[i]-df.loc[j])/range_
            distancia[i][j] = np.sqrt((diff**2).sum())
            D += distancia[i][j] 
    D = D/(n*n)
    alpha = -np.log(0.5)/D
    similitud = np.exp(-alpha*distancia)
    return similitud
def entropia_one_type(similitud):
    entropia_total = 0
    for i in range(similitud.shape[0]-1):
        for j in range(i+1,similitud.shape[0]):
            if similitud[i][j]!=0 and similitud[i][j]!=1:
                entropia_total -= (similitud[i][j]*np.log(similitud[i][j]))+((1-similitud[i][j])*np.log(1-similitud[i][j]))
    return entropia_total

def entropia_total(categorica,numerica,c=True,n=True):
    if c:
        s_c = similitud_categorica(categorica)
    
        entropia_c = entropia_one_type(s_c)
    else:
        entropia_c = 0
    if n : 
        s_n = similitud_numerica(numerica)
    
        entropia_n = entropia_one_type(s_n)
    else:
        entropia_n = 0
    entropia_total = entropia_c + entropia_n
    return entropia_total

Primero se calcula la entropia del dataset total, como se tienen dos tipos de variables, numéricas y categoricas, se usa un tipo de matriz de similitud para cada tipo de variable, se calcula la entropia para los dos tipos de varaibles por separado y se suman al final obteniendo asi una entropia total. 

In [79]:
# F = {X1,X2,X3,X4}
E_f = entropia_total(df_categorica,df_numerica)
# F_f1 = {X2,X3,X4}
E_f1 = entropia_total(df_categorica,df_numerica.iloc[:,1:])
# F_f2 = {X1,X3,X4}
E_f2 = entropia_total(df_categorica,df_numerica.iloc[:,:1])
# F_f3 = {X1,X2,X4}
E_f3 = entropia_total(df_categorica.iloc[:,1:],df_numerica)
# F_f4 = {X1,X2,X3}
E_f4 = entropia_total(df_categorica.iloc[:,:1],df_numerica)

print(f'Entropía del dataset total : {E_f}\n')
print(f'Diferencia de entropia dataset-X1 : {E_f-E_f1}\n')
print(f'Diferencia de entropia dataset-X2 : {E_f-E_f2}\n')
print(f'Diferencia de entropia dataset-X3 : {E_f-E_f3}\n')
print(f'Diferencia de entropia dataset-X4 : {E_f-E_f4}\n')

Entropía del dataset total : 15.451714274519091

Diferencia de entropia dataset-X1 : 1.445137359855849

Diferencia de entropia dataset-X2 : 1.0004990806777698

Diferencia de entropia dataset-X3 : 5.545177444479563

Diferencia de entropia dataset-X4 : 5.545177444479563



Como la diferencia de X2 es la menor, se colola primero en el ranking y se elimina esta dimension ya que es la que menos cambia la entropia al eliminarse. A continuación se implementa un proceso iterativo para crear el ranking.

In [111]:
def ranking_entropia(df):
    
    lista_inicial = list(df.keys())
    ranking = []

    while len(lista_inicial)>1: 
        differences = []
        nueva_df = df.loc[:,lista_inicial]
        numerica= nueva_df.select_dtypes(include='float64')
        categorica = nueva_df.select_dtypes(include='object')
        E_f = entropia_total(categorica,numerica)
        for n in lista_inicial:
            partial_df = nueva_df.loc[:, nueva_df.columns != n]
            numerica= partial_df.select_dtypes(include='float64')
            categorica = partial_df.select_dtypes(include='object')
            if numerica.shape[1]!=0 and categorica.shape[1]!=0:  
                E_f_nueva = entropia_total(categorica,numerica)
                diff = abs(E_f - E_f_nueva)
            elif numerica.shape[1]!=0:
                E_f_nueva = entropia_total(None,numerica,c=False,n=True)
                diff = abs(E_f - E_f_nueva)
            elif categorica.shape[1]!=0:
                E_f_nueva = entropia_total(categorica,None,c=True,n=False)
                diff = abs(E_f - E_f_nueva)
            differences.append(diff)
        element = differences.index(min(differences))
        ranking.append(lista_inicial.pop(element))
    return ranking
        
        

In [115]:
ranking = ranking_entropia(df)
print(f'El ranking de las variables encontrado es, en orden ascendente :{ranking}')

El ranking de las variables encontrado es, en orden ascendente :['X2', 'X3', 'X4']


## 9. 
Al conjunto de datos Adult del repositorio de Machine learning:
1. Convertir todos los atributos numéricos a categóricos utilizando dos estrategias diferentes.
2. Transformar el conjunto de datos de manera que todos los atributos sean numéricos.

## 10.
Escoger un conjunto de datos del repositorio de Machine learning, que tenga varias dimensiones y
que sean numéricas, y aplicar PCA. Describir el nuevo conjunto de datos.